In [1]:
import sys
print(sys.executable)


C:\Users\shady\anaconda3\envs\sackcv\python.exe


In [2]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO

print("Torch:", torch.__version__)
print("OpenCV:", cv2.__version__)
print("NumPy:", np.__version__)
print("YOLO ready")


Torch: 2.9.1+cpu
OpenCV: 4.10.0
NumPy: 1.26.4
YOLO ready


In [36]:
video_path = r"C:\Users\shady\Downloads\رصيف التحميل .mp4"

import os
print("Video exists:", os.path.exists(video_path))


Video exists: True


In [4]:
cap = cv2.VideoCapture(video_path)
print("Video opened:", cap.isOpened())

ret, frame = cap.read()
print("First frame read:", ret)
print("Frame shape:", frame.shape if ret else None)

cap.release()


Video opened: True
First frame read: True
Frame shape: (1088, 1920, 3)


In [5]:
#Taking frames (Screenshots) from the video
import os

output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)

frame_id = 0
saved = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_id % 10 == 0:
        cv2.imwrite(f"{output_dir}/frame_{saved}.jpg", frame)
        saved += 1

    frame_id += 1

cap.release()

print(f"Saved {saved} frames")


Saved 301 frames


In [6]:
!pip install roboflow --upgrade


In [7]:
#python snippet from Roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="HwuKTjrU8pNBeINYbRqR")
project = rf.workspace("shady-aerw4").project("sack-detection-7u1ju")
version = project.version(1)
dataset = version.download("yolov8")
print("Dataset downloaded to:", dataset.location)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to sack-detection-1 in yolov8:: 100%|██████████| 166/166 [00:00<00:00, 289.17it/s]


Dataset downloaded to: C:\Users\shady\sack-detection-1


In [8]:
model = YOLO("yolov8n.pt")

In [9]:
#Training the model
model.train(
    data="sack-detection-1/data.yaml",  # path to Roboflow dataset
    epochs=50,
    imgsz=640,
    batch=8
)


Ultralytics 8.4.7  Python-3.10.19 torch-2.9.1+cpu CPU (Intel Core i5-5200U 2.20GHz)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=sack-detection-1/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002218917DDE0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [27]:
model = YOLO("runs/detect/train/weights/best.pt") # Load trained YOLOv8 model

results = model("frames/frame_10.jpg", conf=0.4) # Practicing on an image
results[0].show()



image 1/1 C:\Users\shady\frames\frame_10.jpg: 384x640 3 sacks, 420.3ms
Speed: 20.6ms preprocess, 420.3ms inference, 65.5ms postprocess per image at shape (1, 3, 384, 640)


In [4]:
# Parameters
output_video_path = "sack_counted.mp4"
COUNT_LINE_Y = 320 
CONF_THRESHOLD = 0.4
TRACKER = "bytetrack.yaml"


# Open video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise Exception("Cannot open video")

# Get frame size and FPS
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Prepare VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Counting variables
counted_ids = set()
total_count = 0
frame_id = 0

# Process video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8 tracking
    results = model.track(
        frame,
        persist=True,
        tracker=TRACKER,
        conf=CONF_THRESHOLD
    )

    # Draw boxes and count
    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        ids = results[0].boxes.id.cpu().numpy()

        for box, track_id in zip(boxes, ids):
            x1, y1, x2, y2 = map(int, box)
            cx = int((x1 + x2)/2)
            cy = int((y1 + y2)/2)

            # Draw bounding box and ID
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, f"ID {int(track_id)}",
                        (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0,255,0), 2)

            # Count only once when crossing the line
            if cy > COUNT_LINE_Y and track_id not in counted_ids:
                counted_ids.add(track_id)
                total_count += 1

    # Draw counting line and total count
    cv2.line(frame, (0, COUNT_LINE_Y), (frame.shape[1], COUNT_LINE_Y), (0,0,255), 2)
    cv2.putText(frame, f"Total sacks: {total_count}",
                (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1.2, (255,0,0), 3)

    # Save frame to output video
    out.write(frame)

# Release everything
cap.release()
out.release()
print("Processing complete!")
print("FINAL COUNT:", total_count)
print("Output video saved as:", output_video_path)

Processing complete!
FINAL COUNT: 54
Output video saved as: sack_counted.mp4
